In [1]:
## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [2]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release-20160523/bin'
#PROGRAM_PATHS = '/scratch/BS/release-BSR/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
#PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = '/scratch/SuperResolution/BRAINSSuperResolution/HCPWorkflows/'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from nipype.interfaces.semtools import *

PYTHON_AUX_PATHS = '/scratch/wmql/tract_querier/tract_querier/'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

In [22]:
# gs: gold standard
# in: input
gs_cst_left_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_July27/WMQL/Baseline_query_cst.left.vtp'
gs_cst_right_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_July27/WMQL/Baseline_query_cst.right.vtp'

sr_cst_left_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_July27/WMQL/WTV_query_cst.left.vtp'
sr_cst_right_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_July27/WMQL/WTV_query_cst.right.vtp'

In [23]:
def ComputeKappaMeasure(gs_bundle,sr_bundle):
    from tract_querier import tract_math, tractography
    import numpy as np
    #
    gs_tractography = tractography.tractography_from_files(gs_bundle)
    sr_tractography = tractography.tractography_from_files(sr_bundle)
    #
    gs_voxels = tract_math.tract_operations.voxelized_tract(gs_tractography, 1.25)
    sr_voxels = tract_math.tract_operations.voxelized_tract(sr_tractography, 1.25)
    #
    all_voxels = np.array(list(gs_voxels.union(sr_voxels)))
    N = (all_voxels.max(0) - all_voxels.min(0)).prod()
    pp = len(gs_voxels.intersection(sr_voxels)) * 1.
    pn = len(gs_voxels.difference(sr_voxels)) * 1.
    nump = len(sr_voxels.difference(gs_voxels)) * 1.
    nn = N - pp - pn - nump
    observed_agreement = (pp + nn) / N
    chance_agreement = (
        (pp + pn) * (pp + nump) + (nn + nump) * (nn + pn)) / (N * N)
    k = (observed_agreement - chance_agreement) / (1 - chance_agreement)
    return k

In [24]:
kappa = ComputeKappaMeasure(gs_cst_left_fn,sr_cst_left_fn)
print(kappa)

0.665780196977


# Now compute kappa measure on a subset of tracts

In [25]:
def ComputeKappaMeasure(gs_bundle,sr_bundle):
    from tract_querier import tract_math, tractography
    import numpy as np
    #
    #####
    def AlignFiber(tract):
        out_tract = tract
        vecs = np.diff(tract, axis=0)
        avg_vec = np.mean(vecs, axis=0)
        dots = np.array([ np.dot(avg_vec,[1,0,0]), np.dot(avg_vec,[0,1,0]), np.dot(avg_vec,[0,0,1]) ])
        idx = np.argmax(np.abs(dots))
        if (dots[idx] < 0):
            out_tract = tract[::-1]
        return out_tract
    #
    def InterpolateFiber(tract):
        new_tract = tract
        # find point distances
        d = np.diff(tract, axis=0)
        pointdists = np.hypot(d[:,0], d[:,1], d[:,2])
        dist_threshold = min(pointdists.sum()/len(pointdists),1)
        offset = 0
        for i in xrange(len(pointdists)):
            if pointdists[i] > dist_threshold:
                new_p = (tract[i+1]+tract[i])/2
                new_tract = np.insert(new_tract,i+1+offset,new_p,axis=0)
                offset += 1
        return new_tract
    #
    def returnBundlePoints(bundle):
        in_tractography = tractography.tractography_from_files(bundle)
        tracts = in_tractography.tracts()
        ## sort all tracts direction to positive coordinate of their dominant orientation
        ## then interpolate tracts to make sure they are approximately equally placed
        for i in xrange(len(tracts)):
            tracts[i] = AlignFiber(tracts[i])
            #tracts[i] = InterpolateFiber(tracts[i])
        #
        tracts_last_quarter = [tracts[i][9*len(tracts[i])/10:] for i in xrange(len(tracts))]
        #
        pts = np.vstack(tracts)
        pts_last_quarter = np.vstack(tracts_last_quarter)
        return pts, pts_last_quarter
    #
    def voxelized_tract(pts, resolution):
        from itertools import izip
        all_points = pts/resolution
        all_points = all_points.round(0).astype(int)
        return set(izip(*(all_points.T)))
    ###
    [gs_pts, gs_pts_last_quarter] = returnBundlePoints(gs_bundle)
    [sr_pts, sr_pts_last_quarter] = returnBundlePoints(sr_bundle)
    #
    #gs_voxels = voxelized_tract(gs_pts, 1.25)
    #sr_voxels = voxelized_tract(sr_pts, 1.25)
    gs_voxels = voxelized_tract(gs_pts_last_quarter, 1.25)
    sr_voxels = voxelized_tract(sr_pts_last_quarter, 1.25)
    #
    all_voxels = np.array(list(gs_voxels.union(sr_voxels)))
    N = (all_voxels.max(0) - all_voxels.min(0)).prod()
    pp = len(gs_voxels.intersection(sr_voxels)) * 1.
    pn = len(gs_voxels.difference(sr_voxels)) * 1.
    nump = len(sr_voxels.difference(gs_voxels)) * 1.
    nn = N - pp - pn - nump
    observed_agreement = (pp + nn) / N
    chance_agreement = (
        (pp + pn) * (pp + nump) + (nn + nump) * (nn + pn)) / (N * N)
    k = (observed_agreement - chance_agreement) / (1 - chance_agreement)
    return k

In [26]:
kappa = ComputeKappaMeasure(gs_cst_left_fn,sr_cst_left_fn)
print(kappa)

0.483362099809


In [ ]:
def ComputeKappaMeasure(gs_bundle,sr_bundle):
    from tract_querier import tract_math, tractography
    import numpy as np
    #####
    def AlignFiber(tract):
        out_tract = tract
        vecs = np.diff(tract, axis=0)
        avg_vec = np.mean(vecs, axis=0)
        dots = np.array([ np.dot(avg_vec,[1,0,0]), np.dot(avg_vec,[0,1,0]), np.dot(avg_vec,[0,0,1]) ])
        idx = np.argmax(np.abs(dots))
        if (dots[idx] < 0):
            out_tract = tract[::-1]
        return out_tract
    #
    def InterpolateFiber(tract):
        new_tract = tract
        # find point distances
        d = np.diff(tract, axis=0)
        pointdists = np.hypot(d[:,0], d[:,1], d[:,2])
        dist_threshold = min(pointdists.sum()/len(pointdists),1)
        offset = 0
        for i in xrange(len(pointdists)):
            if pointdists[i] > dist_threshold:
                new_p = (tract[i+1]+tract[i])/2
                new_tract = np.insert(new_tract,i+1+offset,new_p,axis=0)
                offset += 1
        return new_tract
    #
    def returnBundlePoints(bundle):
        in_tractography = tractography.tractography_from_files(bundle)
        tracts = in_tractography.tracts()
        ## sort all tracts direction to positive coordinate of their dominant orientation
        ## then interpolate tracts to make sure they are approximately equally placed
        for i in xrange(len(tracts)):
            tracts[i] = AlignFiber(tracts[i])
            tracts[i] = InterpolateFiber(tracts[i])
        #
        tracts_last_quarter = [tracts[i][9*len(tracts[i])/10:] for i in xrange(len(tracts))]
        #
        pts = np.vstack(tracts)
        pts_last_quarter = np.vstack(tracts_last_quarter)
        return pts, pts_last_quarter
    #
    def returnKappa(gs_pts,sr_pts):
        mn = np.minimum(gs_pts.min(0), sr_pts.min(0))
        mx = np.maximum(gs_pts.max(0), sr_pts.max(0))
        bins = np.ceil((mx - mn))
        gs_hist = np.array([ np.histogram(gs_pts[:,i], bins=bins[i], density=True, range=(mn[i], mx[i]))[0] for i in xrange(3) ])
        sr_hist = np.array([ np.histogram(sr_pts[:,i], bins=bins[i], density=True, range=(mn[i], mx[i]))[0] for i in xrange(3) ])
        coefs = np.array([ np.sqrt( (gs_hist[i]*sr_hist[i])/(gs_hist[i].sum()*sr_hist[i].sum()) ).sum() for i in xrange(3) ])
        print(coefs)
        return coefs.mean()
    #####
    [gs_pts, gs_pts_last_quarter] = returnBundlePoints(gs_bundle)
    [sr_pts, sr_pts_last_quarter] = returnBundlePoints(sr_bundle)
    #
    coef = returnBhattCoef(gs_pts,sr_pts)
    print(coef)
    coef_last_quarter = returnBhattCoef(gs_pts_last_quarter,sr_pts_last_quarter)
    print(coef_last_quarter)

In [ ]:
ComputeBhattacharyyaCoeficient(gs_cst_left_fn,sr_cst_left_fn)

In [ ]:
ComputeBhattacharyyaCoeficient(gs_cst_right_fn,sr_cst_right_fn)